In [ ]:
# Importing necessary packages
%matplotlib inline
import numpy as np 
import pandas as pd 
import re
import nltk 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/tweets

/content/drive/MyDrive/tweets


In [ ]:
tweets = pd.read_csv("Tweets.csv", sep = ',')
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
# Cleaning the data
text_set = tweets.iloc[:, 10].values #Selecting the 11th column: text
sentiment_set = tweets.iloc[:, 1].values #Selecting the 2nd columnn: airline_sentiment

cleaned_text_set = list()
for input_phrase in range(0, len(text_set)):
    
    clean_text = re.sub(r'\W', ' ', str(text_set[input_phrase])) #Removing spl characters and single characters
    clean_text= re.sub(r'\s+[a-zA-Z]\s+', ' ', clean_text)
    clean_text = re.sub(r'\^[a-zA-Z]\s+', ' ', clean_text) 
    clean_text = clean_text.lower()
    cleaned_text_set.append(clean_text)

In [ ]:
import nltk
nltk.download("stopwords")
input_vector = TfidfVectorizer (max_features=3000, min_df=6, max_df=0.8, stop_words=stopwords.words('english'))
cleaned_text_set = input_vector.fit_transform(cleaned_text_set).toarray()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(cleaned_text_set, sentiment_set, test_size=0.2, random_state=42)

# Logistic Regression
lr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
lr_score = lr.score(X_train, y_train)
print("Logistic Regression Accuracy Score: ", lr_score)

# Support Vector Machine Linear Classification
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)
svc_score = svc.score(X_train, y_train)
print("SVM Classification Accuracy Score: ", svc_score)


Logistic Regression Accuracy Score:  0.8238558743169399
SVM Classification Accuracy Score:  0.858948087431694


,Classifiers,Accuracy on Training Data
0,Support Vector Machine Classification,0.858948
1,Logistic Regression,0.823856


In [ ]:
# create the sub-models
estimators = []

#Defining 2 Logistic Regression Models
model11 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
estimators.append(('logistic1', model11))
model12 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
estimators.append(('logistic2', model12))


#Defining 2 Support Vector Classifiers
model21 = SVC(kernel = 'linear')
estimators.append(('svm1', model21))
model22 = SVC(kernel = 'linear')
estimators.append(('svm2', model22))

# Defining the ensemble model
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train, y_train)
#y_pred = ensemble.predict(X_test)
ensemble_score = ensemble.score(X_train, y_train)
print("Ensemble Score: ", ensemble_score)

Ensemble Score:  0.82599043715847


In [ ]:
final_pred_lr = lr.predict(X_test)
final_pred_svc = svc.predict(X_test)
final_pred_ensemble = ensemble.predict(X_test)
# Accuracy score of the final prediction
print("SVM prediction: ", accuracy_score(y_test, final_pred_svc))
print("LR prediction: ", accuracy_score(y_test, final_pred_lr))
print("Ensemble prediction: ", accuracy_score(y_test, final_pred_ensemble))

SVM prediction:  0.7913251366120219
LR prediction:  0.7940573770491803
Ensemble prediction:  0.7940573770491803
